
warnings.filterwarnings('ignore')

Rutas de archivos

In [55]:
# Robust data directory resolution for both script and notebook
from pathlib import Path
import os

def locate_data_processed(filename='datos_sima_limpios_combined.csv'):
    # Try to use __file__ when available (script execution)
    try:
        start = Path(__file__).resolve().parent.parent
    except NameError:
        start = Path.cwd()

    # Try a sequence of candidate starting points and their parents
    candidates = [start, Path.cwd(), start.parent, Path.cwd().parent]
    # Also include a few ancestor levels for each candidate
    for c in candidates:
        if c is None:
            continue
        for p in [c] + list(c.parents)[:5]:
            dp = p / 'data_processed'
            if dp.exists() and (dp / filename).exists():
                return dp

    # If no folder contains the file, prefer any existing data_processed folder
    for c in candidates:
        dp = Path(c) / 'data_processed'
        if dp.exists():
            return dp

    # Last resort: assume a data_processed at start
    return Path(start) / 'data_processed'

DATA_DIR = locate_data_processed()
INPUT_CSV = DATA_DIR / 'datos_sima_limpios_combined.csv'
OUT_AGG = DATA_DIR / 'datos_aggregados_dia_estacion_franja.csv'

print('DATA_DIR resolved to:', DATA_DIR)
print('INPUT_CSV exists?:', INPUT_CSV.exists())

DATA_DIR resolved to: c:\Users\anala\Documents\Semestre 5\Reto-multivariados-equipo-5\data_processed
INPUT_CSV exists?: True


In [62]:
# Export a normalized calendar CSV so main() can save it
import pandas as pd

def format_calendar_for_csv(calendario, out_path):
    """Escribe un CSV con columnas: estacion, start, end (formato 'YYYY-MM-DD HH:MM:SS').
    Devuelve la ruta escrita como string.
    """
    rows = []
    for est, intervals in calendario.items():
        for s, e in intervals:
            rows.append({'estacion': est, 'start': s, 'end': e})
    df_cal = pd.DataFrame(rows)
    out = Path(out_path)
    out.parent.mkdir(parents=True, exist_ok=True)
    df_cal.to_csv(out, index=False)
    return str(out)

Calendario por estación

In [60]:
CALENDARIO_POR_ESTACION = {
    'CENTRO': [
        ('2023-01-09 00:00:00', '2023-05-12 00:00:00'),
        ('2023-08-07 00:00:00', '2023-12-01 00:00:00'),
        ('2024-01-15 00:00:00', '2024-05-10 00:00:00'),
        ('2024-08-05 00:00:00', '2024-11-29 00:00:00'),
        ('2025-01-06 00:00:00', '2025-05-09 00:00:00')
    ],
    'SUROESTE2': [
        ('2023-01-09 00:00:00', '2023-05-12 00:00:00'),
        ('2023-08-07 00:00:00', '2023-12-06 00:00:00'),
        ('2024-01-10 00:00:00', '2024-05-24 00:00:00'),
        ('2024-08-05 00:00:00', '2024-12-05 00:00:00'),
        ('2025-01-13 00:00:00', '2025-05-23 00:00:00')
    ],
    'SUR': [
        ('2023-01-23 00:00:00', '2023-06-30 00:00:00'),
        ('2023-08-07 00:00:00', '2023-12-06 00:00:00'),
        ('2024-01-22 00:00:00', '2024-06-28 00:00:00'),
        ('2024-08-05 00:00:00', '2024-12-05 00:00:00'),
        ('2025-01-20 00:00:00', '2025-06-27 00:00:00')
    ],
    'NORTE2': [
        ('2023-01-09 00:00:00', '2023-07-01 00:00:00'),
        ('2023-08-23 00:00:00', '2023-12-15 00:00:00'),
        ('2024-01-08 00:00:00', '2024-07-16 00:00:00'),
        ('2024-08-26 00:00:00', '2024-12-18 00:00:00'),
        ('2025-01-09 00:00:00', '2025-07-16 00:00:00')
    ]
}

In [43]:
def load_data(path=INPUT_CSV):
    if not path.exists():
        raise FileNotFoundError(f"Archivo no encontrado: {path}. Ejecute la etapa 1 primero.")
    df = pd.read_csv(path, parse_dates=['date'], dayfirst=False)
    return df

In [44]:
def create_time_vars(df):
    df = df.copy()
    if 'date' not in df.columns:
        raise KeyError('La columna `date` no está presente.')
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.dropna(subset=['date'])
    df['anio'] = df['date'].dt.year
    df['mes'] = df['date'].dt.month
    df['dia'] = df['date'].dt.day
    df['hora'] = df['date'].dt.hour

    try:
        df['dia_semana'] = df['date'].dt.day_name(locale='es_ES')
    except Exception:
        df['dia_semana'] = df['date'].dt.day_name()
    return df

In [45]:
def assign_franja(h):
    # h: integer hour (0-23)
    if h in [7, 8]:
        return 'pico_7_9'
    elif h in [10, 11]:
        return 'ref_10_12'
    else:
        return 'fuera_franja'

In [58]:
def assign_clase_from_calendar(df, calendario=CALENDARIO_POR_ESTACION):
    """Asigna la variable `clase` (1/0) a partir de `calendario`.

    El calendario debe ser un dict: estacion -> list of (start_str, end_str), fechas 'YYYY-MM-DD'.
    Si una estación no está en el calendario, se asigna `clase=0` y se imprime advertencia.
    """
    df = df.copy()
    df['clase'] = 0
    # Preprocesar calendario: convertir a datetimes
    cal_proc = {}
    for est, intervals in calendario.items():
        parsed = []
        for start_s, end_s in intervals:
            try:
                s = pd.to_datetime(start_s)
                e = pd.to_datetime(end_s)
                parsed.append((s, e))
            except Exception:
                continue
        if parsed:
            cal_proc[est] = parsed

    missing_stations = set(df['estacion'].dropna().unique()) - set(cal_proc.keys())
    if missing_stations:
        print(f"Advertencia: las siguientes estaciones no están en el calendario y recibirán clase=0: {sorted(list(missing_stations))}")

    def is_in_class(row):
        est = row['estacion']
        dt = row['date']
        if pd.isna(est) or pd.isna(dt):
            return 0
        # If station not in calendar, it's outside class
        if est not in cal_proc:
            return 0
        # Sólo lunes-viernes
        if dt.weekday() >= 5:
            return 0
        for (s, e) in cal_proc[est]:
            # comparar solo fecha (sin hora)
            if s.normalize() <= dt.normalize() <= e.normalize():
                return 1
        return 0

    df['clase'] = df.apply(is_in_class, axis=1)
    return df

In [73]:
def filter_and_aggregate(df):
    """Selecciona las horas objetivo (7,8,10,11), asigna `franja_horaria` y devuelve
    filas con `date`, `anio`,`mes`,`dia`,`hora`,`estacion`,`franja_horaria`,`parametro`,`valor`,`clase`.

    No se agregan medias/medianas/contajes — se preserva cada registro horario relevante.
    """
    df = df.copy()
    # Mantener sólo parámetros esperados
    expected = ['NO2', 'CO', 'PM2.5', 'PM10', 'O3']
    df = df[df['parametro'].isin(expected)]

    # Filtrar a las horas exactas solicitadas
    target_hours = [7, 8, 10, 11]
    df = df[df['hora'].isin(target_hours)]

    # Asignar franja según hora
    df['franja_horaria'] = df['hora'].apply(assign_franja)

    # Seleccionar y devolver columnas deseadas, preservando `valor` y `clase` si existe
    cols = ['date','anio','mes','dia','hora','estacion','franja_horaria','parametro','valor','clase']
    # Asegurar que las columnas existan en el df
    existing = [c for c in cols if c in df.columns]
    out = df[existing].copy()

    # Orden consistente
    final_cols = [c for c in cols if c in out.columns]
    return out[final_cols]

In [49]:
def save_aggregated(agg_df, path=OUT_AGG):
    path.parent.mkdir(parents=True, exist_ok=True)
    agg_df.to_csv(path, index=False)
    print(f"✓ Agregados guardados en: {path}")

In [50]:
def compute_acf(series, nlags=30):
    if acf is None:
        raise ImportError('statsmodels no disponible. Instale statsmodels para usar ACF.')
    series = series.dropna()
    return acf(series, nlags=nlags, fft=False)

In [51]:
def compute_ljungbox(series, lags=[10]):
    if acorr_ljungbox is None:
        raise ImportError('statsmodels no disponible. Instale statsmodels para Ljung-Box.')
    series = series.dropna()
    res = acorr_ljungbox(series, lags=lags, return_df=True)
    return res

In [52]:
def compute_durbin_watson(residuals):
    if durbin_watson is None:
        raise ImportError('statsmodels no disponible. Instale statsmodels para Durbin-Watson.')
    return durbin_watson(residuals)

In [ ]:
def main():
    print('\n== Parte 2: creación de variables y agregación ==')
    print(f'Leyendo: {INPUT_CSV}')
    df = load_data(INPUT_CSV)
    print(f'Registros cargados: {len(df):,}')

    df = create_time_vars(df)
    print('Variables temporales creadas: anio, mes, dia, hora, dia_semana')

    print('Asignando variable `clase` según calendario...')
    if not CALENDARIO_POR_ESTACION:
        print('  ATENCIÓN: `CALENDARIO_POR_ESTACION` está vacío. Edite el script y agregue los periodos por estación antes de ejecutar para obtener `clase=1` cuando corresponda.')
    # Exportar versión normalizada del calendario con formato 'YYYY-MM-DD HH:MM:SS'
    try:
        # Usar DATA_DIR (siempre disponible en notebook y script)
        cal_export_path = DATA_DIR / 'calendario_estaciones_normalized.csv'
        norm_out = format_calendar_for_csv(CALENDARIO_POR_ESTACION, cal_export_path)
        if norm_out:
            print(f'  Calendario normalizado guardado en: {norm_out}')
    except Exception as e:
        print(f'  No fue posible exportar calendario normalizado: {e}')

    df = assign_clase_from_calendar(df, calendario=CALENDARIO_POR_ESTACION)

    print('Filtrando franjas y agregando por día-estación-franja...')
    agg = filter_and_aggregate(df)
    save_aggregated(agg)

    print('\nTareas siguientes sugeridas:')
    print('- Rellenar `CALENDARIO_POR_ESTACION` con los intervalos reales por estación.')
    print('- Ejecutar nuevamente para obtener `clase` correcta y luego aplicar pruebas ACF/Ljung-Box/Durbin-Watson con las funciones disponibles.')

In [74]:
main()


== Parte 2: creación de variables y agregación ==
Leyendo: c:\Users\anala\Documents\Semestre 5\Reto-multivariados-equipo-5\data_processed\datos_sima_limpios_combined.csv
Registros cargados: 1,804,203
Registros cargados: 1,804,203
Variables temporales creadas: anio, mes, dia, hora, dia_semana
Asignando variable `clase` según calendario...
  DEBUG: DATA_DIR=c:\Users\anala\Documents\Semestre 5\Reto-multivariados-equipo-5\data_processed
  DEBUG: format_calendar_for_csv defined? True
  DEBUG: cal_export_path=c:\Users\anala\Documents\Semestre 5\Reto-multivariados-equipo-5\data_processed\calendario_estaciones_normalized.csv
  Calendario normalizado guardado en: c:\Users\anala\Documents\Semestre 5\Reto-multivariados-equipo-5\data_processed\calendario_estaciones_normalized.csv
Variables temporales creadas: anio, mes, dia, hora, dia_semana
Asignando variable `clase` según calendario...
  DEBUG: DATA_DIR=c:\Users\anala\Documents\Semestre 5\Reto-multivariados-equipo-5\data_processed
  DEBUG: form

In [ ]:
# Cargar el CSV agregado y extraer componentes de fecha
agg_df = pd.read_csv(OUT_AGG, parse_dates=['date'])

# Ya tenemos anio, mes, dia en el CSV. Vamos a verificar y agregar hora si falta
print("Primeras 5 filas del agregado:")
print(agg_df.head())
print(f"\nColumnas disponibles:\n{agg_df.columns.tolist()}")
print(f"\nForma del dataset: {agg_df.shape}")

# Extraer hora de la columna 'date' si no está ya
if 'hora' not in agg_df.columns:
    agg_df['hora'] = agg_df['date'].dt.hour
    print("\n✓ Columna 'hora' extraída de 'date'")

# Ver distribuición de datos
print("\n== Resumen ==")
print(f"Estaciones: {agg_df['estacion'].unique()}")
print(f"Parámetros: {agg_df['parametro'].unique()}")
print(f"Franjas: {agg_df['franja_horaria'].unique()}")
print(f"Años: {sorted(agg_df['anio'].unique())}")
print(f"Rango de fechas: {agg_df['date'].min()} a {agg_df['date'].max()}")
